# Assignment 1: Web Scraping

## Objective

Data scientists often need to crawl data from websites and turn the crawled data (HTML pages) to structured data (tables). Thus, web scraping is an essential skill that every data scientist should master. In this assignment, you will learn the followings:


* How to use [requests](http://www.python-requests.org/en/master/) to download HTML pages from a website?
* How to select content on a webpage with [lxml](http://lxml.de/)? 

You can either use Spark DataFrame or [pandas.DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) to do the assignment. In comparison, pandas.DataFrame has richer APIs, but is not good at distributed computing.

## Preliminary

If this is your first time to write a web scraper, you need to learn some basic knowledge of HTML, DOM, and XPath. I found that this is a good resource: [https://data-lessons.github.io](https://data-lessons.github.io/library-webscraping-DEPRECATED/). Please take a look at

* [Selecting content on a web page with XPath
](https://data-lessons.github.io/library-webscraping-DEPRECATED/xpath/)
* [Web scraping using Python: requests and lxml](https://data-lessons.github.io/library-webscraping-DEPRECATED/04-lxml/). 

Please let me know if you find a better resource. I'll share it with the other students.

## Overview

Imagine you are a data scientist working at SFU. One day, you want to analyze CS faculty data and answer two interesting questions:

1. Who are the CS faculty members?
2. What are their research interests?

To do so, the first thing is to figure out what data to collect.

## Task 1: SFU CS Faculty Members

You find that there is a web page in the CS school website, which lists all the faculty members as well as their basic information. 

In Task 1, your job is to write a web scraper to extract the faculty information from this page: [https://www.sfu.ca/computing/people/faculty.html](https://www.sfu.ca/computing/people/faculty.html).




### (a) Crawling Web Page

A web page is essentially a file stored in a remote machine (called web server). You can use [requests](http://www.python-requests.org/en/master/) to open such a file and read data from it. Please complete the following code to download the HTML page and save it as a text file (like [this](./faculty.txt)). 

In [1]:
import requests
response = requests.get('https://www.sfu.ca/computing/people/faculty.html')
filename = 'sfufaculty.txt'
with open(filename,'w') as fd:
    fd.write(response.text)

# 1. Download the webpage
# 2. Save it as a text file (named faculty.txt)

### (b) Extracting Structured Data

An HTML page follows the Document Object Model (DOM). It models an HTML page as a tree structure wherein each node is an object representing a part of the page. The nodes can be searched and extracted programmatically using XPath. Please complete the following code to transform the above HTML page to a CSV file (like [this](./faculty_table.csv)). 

In [1]:
import lxml.html 
import requests
import lxml.html
from bs4 import BeautifulSoup
import pandas as pd
import re


filename = 'sfufaculty.txt'

with open(filename) as f:
	content = f.read()


html_soup = BeautifulSoup(content,'html.parser')

domain = 'http://www.sfu.ca'
#print(type(html_soup))
texts = html_soup.find_all('div', class_='text')
names=[]
ranks=[]
area=[]
profile =[]
homepage=[]
#print len(texts)
for x in texts:
	try:
		whole = x.h4.get_text()

		name = whole.split(',')[0]  #finding the name of preofessors
		names.append(name)
		rank = whole.split(',')[1]
		rank2 = rank.split('\n')[0] # finding the ranks of professor
		ranks.append(rank2)

		
		for b_tag in x.find_all('b'): #finding the area of research
			areas1 = b_tag.next_sibling
		area.append(areas1)


		links = x.find_all('a',string='Profile & Contact Information') or x.find_all('a', string='Profile & Contact Information'u'\xa0')
		for o in links:
			link1 = o.get('href')
			if link1.startswith('http://'):
				link2 = link1
			else:
				link2 = domain + link1
			#print(link1)
		profile.append(link2)



		hpage = x.find_all('a',string = 'Home Page')
		#print (hpage)
		if hpage:
			for h in hpage:
				link4 = h.get('href')
				#print(link4)		
				homepage.append(link4)
		else:
			homepage.append(':')


		

	
	

	except AttributeError:
		pass



#print('name:',names)
#print('rank:',ranks)
#print (homepage)

df = pd.DataFrame(names,columns=['Name'])
df['Rank']=ranks
df['area'] = area
df['profile'] = profile
df['homepage'] = homepage

#print(df)
df.to_csv('faculty_table.csv',encoding = 'utf-8',index=True)

# 1. Open faculty.txt
# 2. Parse the HTML page as a tree structure
# 3. Extract related content from the tree using XPath
# 4. Save the extracted content as an csv file (named faculty_table.csv)

## Task 2: Research Interests

Suppose you want to know the research interests of each faculty. However, the above crawled web page does not contain such information. 

### (a) Crawling Web Page

You notice that such information can be found on the profile page of each faculty. For example, you can find the research interests of Dr. Jiannan Wang from [http://www.sfu.ca/computing/people/faculty/jiannanwang.html](http://www.sfu.ca/computing/people/faculty/jiannanwang.html). 


Please complete the following code to download the profile pages and save them as text files. There are 60 faculties, so you need to download 60 web pages in total. 

In [1]:
import requests
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

dirName = 'profilepages'
#profile1=[]
filename=[]

data = pd.read_csv('faculty_table.csv')
#print(data)
profile1 = data['profile']
#for p in profile1:
        #print(p)
name = data['Name'].values
index =0
for p in profile1:
        response=requests.get(p)
        #soup = BeautifulSoup(response.content,'lxml')
        with open((name[index]+'.txt'),'w') as f:
                f.write(response.text)
        index = index + 1



# 1. Download the profile pages of 60 faculties
# 2. Save each page as a text file


### (b) Extracting Structured Data

Please complete the following code to extract the research interests of each faculty, and generate a file like [this](./faculty_more_table.csv). 

In [1]:
from lxml import etree
import lxml.html
import pandas as pd
from bs4 import BeautifulSoup
import re

# 1. Open each text file and parse it as a tree structure

df = pd.read_csv("faculty_table.csv")
names = df['Name'].values
lc = []


element_list=[]*60
index = 0
for val in names:

    with open(val+".txt",'r') as f:
        content = f.read()
    html_soup = BeautifulSoup(content,'html.parser')
    texts = html_soup.find_all('h2',text = re.compile('Research interests')) or html_soup.find_all('h2',text = re.compile('Research Interests'))
    if not texts:
        lc.append('[]')
    else:
        lc5=[]
        for x in texts:
            p = x.next_sibling.next_sibling
            list_element = p.find_all('li')
            for i in list_element:
                lc1 = i.text.strip()
                lc5.append(lc1)
            lc.append(lc5)
        
    index+=1
           
           
# 3. Add the extracted content to faculty_table.csv
df['research_interests'] = lc
df.to_csv('faculty_table.csv', sep=',', encoding='utf-8', index=False) 

# 4. Generate a new CSV file, named faculty_more_table.csv
df.to_csv('faculty_more_table.csv', sep=',', encoding='utf-8', index=False)

# 1. Open each text file and parse it as a tree structure 
# 2. Extract the research interests from each tree using XPath
# 3. Add the extracted content to faculty_table.csv
# 4. Generate a new CSV file, named faculty_more_table.csv

## Submission

Complete the code in this [notebook](A1.ipynb), and submit it to the CourSys activity `Assignment 1`.